Dataset for training model is taken from: https://github.com/prigrecov/authorship_profiling/tree/9b5f2220bd823d0e5a1aa4589bffe593a3f16a7c

In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive
/content/drive/MyDrive/Colab Notebooks/Ziwen 


In [ ]:
# Packages for data processing
import re
import pandas as pd
import string
import spacy
import numpy as np

import nltk.data
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import RegexpTokenizer
from nltk import word_tokenize   
from nltk.stem import WordNetLemmatizer
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer

from sklearn import svm
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import BernoulliNB
from sklearn.svm import LinearSVC, SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from statistics import mode

In [ ]:
nltk.download('stopwords')
stop_word = stopwords.words('english')

word_tokenizer = RegexpTokenizer(r"[A-Za-z]\w+(?:[-'?]\w+)?")
 
stemmer = PorterStemmer()
lemmatizer = WordNetLemmatizer()

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
train_df=pd.read_csv("/content/drive/MyDrive/Colab Notebooks/final_year_project/run_tweet_features/4)/gender_classification/train_labels.csv") 

In [ ]:
#display xml
with open('/content/drive/MyDrive/Colab Notebooks/final_year_project/run_tweet_features/4/gender_classification/data1/d7d392835f50664fc079f0f388e147a0.xml','r',encoding='cp437') as in_file:
    xml_first_data = in_file.read()


<author lang="en">
	<documents>
		<document><![CDATA[@CSIFERROSCAN youch! Good things to know! Is that sort of stuff repairable?]]></document>
		<document><![CDATA[Succumbed to FOMO and bought GNR tickets. Remember asking parents to buy GNR Lies album as a young boy because it had boobies inside. ≡ƒÿé]]></document>
		<document><![CDATA[Brown eye Broom a cool 73 then to the rescue to clear out some broken window in Wellington @TheACCnz https://t.co/eoYmdswYX5]]></document>
		<document><![CDATA[Shout out to Auckland tennis fans who get to sleep in this morning but worth it to watch the ≡ƒÉÉ win #18 #Federer #needcoffee]]></document>
		<document><![CDATA[@nickyoung210 Someone had some balls to come up with that ≡ƒÿé]]></document>
		<document><![CDATA[@okcthunder Hey guys, are there any more releases coming of the 2016 Christmas day jerseys?]]></document>
		<document><![CDATA[@JulieReddish Yeah but curiosity got the better of me!! ≡ƒæÇ]]></document>
		<document><![CDATA[@JulieReddish I rew

In [ ]:
def preprocess(xml_data):
    pattern_text = pattern_text = r'<!\[CDATA\[(.*?)\]\]>' #extracting the sentences
 
    xml_text = re.findall(pattern_text, xml_data,  flags=re.DOTALL)

    for i in range(0,len(xml_text)):
    
        # Remove of unwanted characters
        xml_text[i] = re.sub('@[^\s]* ', '',  xml_text[i]) 
        xml_text[i] = re.sub(' https://.*', '', xml_text[i])
        xml_text[i] = re.sub('[^A-Za-z0-9\s]*', '', xml_text[i])

        xml_text[i] = xml_text[i].lower()
    
    xml_text = ' '.join(xml_text)
    return(xml_text)

In [ ]:

train_vec = []
pre_processed_data=[]
def read_xml(file_name):
    with open(file_name,'r',encoding='cp437') as in_file:
        xml_data = in_file.read()
        
        pre_processed_dt = preprocess(xml_data)
        pre_processed_data.append(pre_processed_dt)
        return(pre_processed_dt)

In [ ]:
#Processing the train dataset 
train_df['id_link'] = ''
train_df['preprocessed'] = ''

for i in range(0,len(train_df)):
    train_df['id_link'][i]='/content/drive/MyDrive/Colab Notebooks/final_year_project/run_tweet_features/4/gender_classification/data1/' +train_df['id'][i]+'.xml'

    preprocessed = read_xml(train_df['id_link'][i])
    train_df['preprocessed'][i]=preprocessed

In [ ]:
train_df.head()


,id,gender,id_link,preprocessed
0,d7d392835f50664fc079f0f388e147a0,male,/content/drive/MyDrive/Colab Notebooks/Ziwen /...,youch good things to know is that sort of stuf...
1,ee40b86368137b86f51806c9f105b34b,female,/content/drive/MyDrive/Colab Notebooks/Ziwen /...,donald the menace thankscomey a return to nat...
2,919bc742d9a22d65eab1f52b11656cab,male,/content/drive/MyDrive/Colab Notebooks/Ziwen /...,this seems super sketch too good to be true\n...
3,15b97a08d65f22d97ca685686510b6ae,female,/content/drive/MyDrive/Colab Notebooks/Ziwen /...,just some texts with my dad about our saturday...
4,affa98421ef5c46ca7c8f246e0a134c1,female,/content/drive/MyDrive/Colab Notebooks/Ziwen /...,irrevocably love this talented human and so pr...


In [ ]:
global x_train, y_train

train_text = train_df['preprocessed'].tolist() 
train_label = train_df.gender.tolist() 

stopwords_fin = stop_word+list(string.punctuation)

import nltk
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('omw-1.4')
# Defining the tokenizer
class LemmaTokenizer(object):
    def __init__(self):
        self.wnl=WordNetLemmatizer()
    def __call__(self,doc):
        
        output = ''.join(re.findall('[a-zA-Z\s]+',doc))
        return [self.wnl.lemmatize(t) for t in word_tokenize(output)]

# TF-IDF vectorization
vectorizer=TfidfVectorizer(analyzer='word',input='content',
                           lowercase=True,
                           token_pattern='(?u)\\b\\w\\w+\\b',
                           min_df=0.05,
                           max_df=0.95,
                           ngram_range=(1,3),
                           tokenizer=LemmaTokenizer(),
                           stop_words=stopwords_fin)


x_train=vectorizer.fit_transform(train_text)
y_train=np.asarray(train_label)

#https://stackoverflow.com/questions/61916096/word-cloud-built-out-of-tf-idf-vectorizer-function
feature_names = vectorizer.get_feature_names()
dense = x_train.todense()
lst1 = dense.tolist()
df = pd.DataFrame(lst1, columns=feature_names)
df.T.sum(axis=1)

In [ ]:
hashtags = ['#VoteThemOut',
            # '#ToryScumOut',
            'F1',
            'Lisa',
            '#EnoughIsEnough',
            '#EnergyPrices',
            '#iOS16',
            '#taiwan',
            '#onepiece',
            '#CostOfLivingCrisis',
            '#GetBackToWorkYouFatPonce', 
            # '#ClosingCeremony',
            '#BookLoversDay',
            '#biden']
# hashtags = ['F1']
for hash in hashtags:
        test_df = pd.read_csv(f'/content/drive/MyDrive/Colab Notebooks/final_year_project/run_tweet_features/3/{hash}_tweet_features_with_offensiveirony.csv')
        test_text = test_df['text'].tolist()
        global x_test
        x_test=vectorizer.transform(test_text)

        #LR
        lr = LogisticRegression()
        model_name = lr.__class__.__name__
        lr.fit(x_train, y_train)
        lr_predict=lr.predict(x_test)

        #Linear SVC
        svc = SVC(C=0.8,gamma=1,degree=3,kernel='linear')
        model_name = svc.__class__.__name__
        svc.fit(x_train, y_train)
        svc_predict=svc.predict(x_test)

        #Random Forest
        rf = RandomForestClassifier(n_estimators=50, criterion='gini', max_depth=None,min_samples_split=2, min_samples_leaf=1, min_weight_fraction_leaf=0.0, max_features='auto', max_leaf_nodes=None,bootstrap=True, oob_score=False, n_jobs=1, random_state=100, verbose=0, warm_start=False,class_weight=None)
        model_name = rf.__class__.__name__
        rf.fit(x_train, y_train)
        rf_predict=rf.predict(x_test)

        #https://www.geeksforgeeks.org/python-statistics-mode-function/#:~:text=Python%20statistics%20%7C%20mode%20function,most%20likely%20to%20be%20sampled.&text=Syntax%20%3A%20mode(%5Bdata%2Dset,numbers%20as%20well%20as%20Strings.
        #Finding the mode (most number of occcuring ) of the predictions into a list
        best_predictions = [mode(prediction) for prediction in zip(lr_predict,svc_predict,rf_predict)]
        print(best_predictions)

        test_df['gender'] =  best_predictions
        test_df['gender'] = test_df['gender'].map({'male': 1, 'female': 0})
        test_df.head()
        test_df.to_csv(f'/content/drive/MyDrive/Colab Notebooks/final_year_project/run_tweet_features/4/{hash}_tweet_features_with_gender.csv')

LogisticRegression
SVC
RandomForestClassifier
['male', 'male', 'female', 'female', 'male', 'male', 'female', 'male', 'female', 'female', 'female', 'female', 'male', 'male', 'female', 'female', 'female', 'female', 'male', 'female', 'male', 'male', 'female', 'male', 'male', 'male', 'female', 'female', 'female', 'male', 'male', 'male', 'male', 'male', 'male', 'male', 'male', 'female', 'female', 'male', 'male', 'female', 'male', 'female', 'female', 'female', 'male', 'female', 'female', 'female', 'male', 'female', 'male', 'male', 'female', 'female', 'female', 'male', 'female', 'female', 'female', 'male', 'female', 'male', 'female', 'male', 'female', 'female', 'male', 'female', 'male', 'male', 'male', 'female', 'male', 'female', 'female', 'female', 'female', 'female', 'male', 'female', 'female', 'female', 'female', 'male', 'female', 'female', 'female', 'male', 'female', 'male', 'male', 'female', 'female', 'female', 'female', 'male', 'male', 'male', 'male', 'male', 'female', 'male', 'female',

In [ ]:
# print(type(x_test))

# Test for accuracy

In [ ]:
test_df=pd.read_csv("/content/drive/MyDrive/Colab Notebooks/final_year_project/run_tweet_features/4/gender_classification/test_labels.csv") # reading test data


In [ ]:
test_df['id_link'] = ''
test_df['clean_text'] = ''

for i in range(0,len(test_df)):
    test_df['id_link'][i]='/content/drive/MyDrive/Colab Notebooks/final_year_project/run_tweet_features/4/gender_classification/data1/' +test_df['id'][i]+'.xml'
    # Calling function to read XML files
    preprocessed = read_xml(test_df['id_link'][i])
    test_df['preprocessed'][i]=preprocessed

In [ ]:
test_text = test_df.preprocessed.tolist() 

In [ ]:
x_test=vectorizer.transform(test_text)

In [ ]:
x_test

<500x1862 sparse matrix of type '<class 'numpy.float64'>'
	with 140825 stored elements in Compressed Sparse Row format>

In [ ]:
from sklearn.metrics import f1_score


#LR
lr = LogisticRegression()
model_name = lr.__class__.__name__
#fit the model to the data
lr.fit(x_train, y_train)
print(model_name)
# Do the prediction
lr_predict=lr.predict(x_test)


#SVC
svc = SVC(C=0.8,gamma=1,degree=3,kernel='linear')
model_name = svc.__class__.__name__
#Fit the model to the data
svc.fit(x_train, y_train)
print(model_name)
# Do the predictions
svc_predict=svc.predict(x_test)

#Random Forest
rf = RandomForestClassifier(n_estimators=50, criterion='gini', max_depth=None,min_samples_split=2, min_samples_leaf=1, min_weight_fraction_leaf=0.0, max_features='auto', max_leaf_nodes=None,bootstrap=True, oob_score=False, n_jobs=1, random_state=100, verbose=0, warm_start=False,class_weight=None)
model_name = rf.__class__.__name__
#Fit the model for the data
rf.fit(x_train, y_train)
print(model_name)
# Do the predictions
rf_predict=rf.predict(x_test)

#https://www.geeksforgeeks.org/python-statistics-mode-function/#:~:text=Python%20statistics%20%7C%20mode%20function,most%20likely%20to%20be%20sampled.&text=Syntax%20%3A%20mode(%5Bdata%2Dset,numbers%20as%20well%20as%20Strings.
#Finding the mode (most number of occcuring ) of the predictions into a list
best_predictions = [mode(prediction) for prediction in zip(lr_predict,svc_predict,rf_predict)]

print(len(best_predictions))

LogisticRegression
SVC
RandomForestClassifier
500


In [ ]:
print(best_predictions)


['male', 'female', 'male', 'male', 'male', 'male', 'male', 'male', 'male', 'male', 'female', 'male', 'male', 'female', 'female', 'female', 'female', 'male', 'male', 'female', 'male', 'male', 'male', 'male', 'female', 'female', 'male', 'female', 'female', 'male', 'female', 'male', 'male', 'female', 'female', 'female', 'male', 'female', 'male', 'female', 'male', 'female', 'male', 'female', 'female', 'male', 'female', 'male', 'male', 'female', 'male', 'female', 'female', 'female', 'male', 'male', 'female', 'male', 'female', 'male', 'female', 'female', 'female', 'female', 'male', 'male', 'male', 'female', 'female', 'female', 'male', 'male', 'female', 'male', 'male', 'female', 'male', 'male', 'female', 'male', 'female', 'male', 'female', 'female', 'female', 'male', 'male', 'female', 'female', 'male', 'female', 'male', 'female', 'female', 'male', 'female', 'male', 'female', 'male', 'female', 'male', 'male', 'male', 'female', 'female', 'male', 'female', 'female', 'male', 'male', 'female', 'fe

In [ ]:
predictions

['male',
 'female',
 'male',
 'male',
 'male',
 'male',
 'male',
 'male',
 'female',
 'female',
 'male',
 'male',
 'male',
 'male',
 'female',
 'female',
 'male',
 'male',
 'male',
 'female',
 'male',
 'male',
 'male',
 'male',
 'male',
 'female',
 'male',
 'female',
 'female',
 'male',
 'female',
 'male',
 'male',
 'male',
 'female',
 'female',
 'female',
 'female',
 'male',
 'female',
 'female',
 'female',
 'male',
 'female',
 'female',
 'male',
 'female',
 'male',
 'male',
 'female',
 'male',
 'female',
 'female',
 'female',
 'male',
 'female',
 'female',
 'female',
 'female',
 'male',
 'female',
 'female',
 'female',
 'female',
 'female',
 'male',
 'male',
 'female',
 'female',
 'female',
 'female',
 'male',
 'female',
 'male',
 'male',
 'female',
 'male',
 'male',
 'female',
 'male',
 'female',
 'male',
 'female',
 'female',
 'female',
 'male',
 'male',
 'female',
 'female',
 'female',
 'female',
 'female',
 'female',
 'female',
 'male',
 'female',
 'male',
 'female',
 'male',
 'f

In [ ]:
test_df=pd.read_csv("/content/drive/MyDrive/Colab Notebooks/final_year_project/run_tweet_features/4/test_labels.csv")
#Store the gender labels in the predictions list
predictions=test_df.gender.to_list()

from sklearn.metrics import recall_score, precision_score, f1_score,accuracy_score
print(len(predictions))
print("Accuracy_score of ensemble model:",accuracy_score(predictions,best_predictions))
print( 'F1 score:', f1_score(predictions,best_predictions, average="binary", pos_label="male"))
print( 'Recall:', recall_score(predictions,best_predictions, average="binary", pos_label="male"))
print( 'Precision:', precision_score(predictions,best_predictions, average="binary", pos_label="male"))



500
Accuracy_score of ensemble model: 0.816
F1 score: 0.8203125000000001
Recall: 0.8467741935483871
Precision: 0.7954545454545454


In [ ]:
# The gender labels which are predicted are stored in a dataframe and then exported to a csv file

#Output dataframe with the gender labels and document ids
output=pd.DataFrame(list(zip(test_df['id'].tolist(),best_predictions)),columns=['id','gender'])
#male and female are converted to 1 & 0
output['gender'] = output['gender'].map({'male': 1, 'female': 0})
output.head()

,id,gender
0,1559034163583504387,0
1,1559034163604381696,1
2,1559034163625447425,0
3,1559034244932018179,1
4,1559034330759962624,1
